# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch_scatter import scatter

import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures, line_graph
from torch_geometric.nn import GATv2Conv
from torch_geometric.utils import softmax

import numpy as np
import pandas as pd
import utils

# Data Overview

In [3]:
#pd.describe_option('display.width')
pd.set_option('display.width', 200)
cnv_data = pd.read_csv('data/cnv_data_brca.tsv', index_col=0, header=0, sep='\t')
print(cnv_data.head())

      A1BG   A1CF    A2M  A2ML1  A4GALT  A4GNT   AAAS   AACS  AADAC  AADACL2  ...   ZXDA   ZXDB   ZXDC  ZYG11A  ZYG11B    ZYX  ZZEF1   ZZZ3        sample  icluster_cluster_assignment
610  0.000 -0.468  0.186  0.186  -0.596  0.259 -0.503 -0.470  0.259    0.259  ...  0.274  0.274  0.259   0.069   0.069  0.209 -0.567  0.249  TCGA-3C-AAAU                           19
611  0.096 -0.461 -0.007 -0.007  -0.533  0.476  0.000  0.000  0.476    0.476  ...  0.822  0.822  0.476  -0.440  -0.440  0.454 -0.525 -0.440  TCGA-3C-AALI                            2
612  0.005  0.034  0.025  0.025  -0.692 -0.136 -0.319  0.371 -0.136   -0.136  ... -0.063 -0.063 -0.137  -0.324  -0.324 -0.003 -0.694 -0.324  TCGA-3C-AALJ                           17
613 -0.014  0.014 -0.002 -0.002   0.020 -0.058  0.000  0.000 -0.058   -0.058  ...  0.011  0.011 -0.058  -0.023  -0.023  0.014 -0.644 -0.023  TCGA-3C-AALK                            2
749  0.295  0.009 -0.344 -0.344  -0.028 -0.003 -0.015 -0.015 -0.003   -0.003  ... -0.

In [4]:
brca_subtype = pd.read_csv('data/brca_subtype.csv', header=0)
print(brca_subtype[:10])

        patient subtype
0  TCGA-3C-AAAU    LumA
1  TCGA-3C-AALI    Her2
2  TCGA-3C-AALJ    LumB
3  TCGA-3C-AALK    LumA
4  TCGA-4H-AAAK    LumA
5  TCGA-5L-AAT0    LumA
6  TCGA-5L-AAT1    LumA
7  TCGA-5T-A9QA    LumB
8  TCGA-A1-A0SD    LumA
9  TCGA-A1-A0SE    LumA


In [5]:
expression_data = pd.read_csv('data/expression_data_brca.tsv', sep='\t', header=0)
print(expression_data.shape)
print(expression_data.head())
# print(expression_data.iloc[5, 3])
# This would be the expression of the 5th gene in the 3rd sample

(981, 17947)
   Unnamed: 0      A1BG      A1CF       A2M     A2ML1    A4GALT     A4GNT      AAAS      AACS     AADAC  ...      ZXDA      ZXDB      ZXDC    ZYG11A    ZYG11B       ZYX     ZZEF1      ZZZ3  \
0         610  0.446721  0.000000  0.731850  0.073185  0.357728  0.191452  0.547424  0.590749  0.000000  ...  0.411593  0.584309  0.626464  0.469555  0.599532  0.689696  0.637588  0.597775   
1         611  0.449630  0.000000  0.733637  0.137735  0.416050  0.035857  0.557769  0.552647  0.060330  ...  0.337507  0.501423  0.591349  0.434832  0.525896  0.707456  0.590211  0.493455   
2         612  0.500000  0.053265  0.750859  0.000000  0.525200  0.000000  0.544101  0.605956  0.053265  ...  0.296678  0.518900  0.549255  0.479954  0.518900  0.710767  0.565865  0.515464   
3         613  0.405232  0.000000  0.716498  0.075280  0.479979  0.000000  0.512547  0.520555  0.104645  ...  0.310198  0.468767  0.521089  0.398292  0.493860  0.665777  0.513081  0.505072   
4         749  0.433012  0.

In [6]:
# Check number of unique samples (in sample column) and ensure they match the rest of the data
print("Number of unique samples in CNV data:", cnv_data['sample'].nunique())
print("Number of unique samples in expression data:", expression_data['sample'].nunique()) 

Number of unique samples in CNV data: 981
Number of unique samples in expression data: 981


In [7]:
brca_subtype = pd.read_csv('data/brca_subtype.csv', sep=',', header=0)
print(brca_subtype['patient'].nunique())

# Check overlap between 'patient' in brca_subtype and 'sample' in cnv_data
overlap = set(brca_subtype['patient']).intersection(set(cnv_data['sample']))
print(f"Number of overlapping samples: {len(overlap)}")
print(f"Example overlapping IDs: {list(overlap)[:10]}")

1045
Number of overlapping samples: 981
Example overlapping IDs: ['TCGA-A8-A06O', 'TCGA-AR-A0U3', 'TCGA-AN-A0AJ', 'TCGA-B6-A0X4', 'TCGA-BH-A18S', 'TCGA-A2-A0YH', 'TCGA-A8-A08A', 'TCGA-E2-A3DX', 'TCGA-A2-A0EP', 'TCGA-A2-A1G0']


In [13]:
gene_interactions = np.load('data/adj_matrix_biogrid.npz')
bg_nonnull = pd.read_csv('data/biogrid_non_null.csv', sep=',', header=0)

for item in gene_interactions:
    print(item)
    print(gene_interactions[item])
    print(gene_interactions[item].shape)

# ar = gene_interactions['data']

# stuff = pd.DataFrame(ar)
# print(stuff.head())
# print(stuff.shape)

indices
[ 2156 16356 17063 ... 17853 17889 17913]
(868686,)
indptr
[     0      4      4 ... 868568 868577 868686]
(17945,)
format
b'csr'
()
shape
[17944 17944]
(2,)
data
[1. 1. 1. ... 1. 2. 1.]
(868686,)


In [9]:
from scipy.sparse import csr_matrix
adj_matrix = csr_matrix(
    (gene_interactions['data'],
     gene_interactions['indices'],
     gene_interactions['indptr']),
    shape=tuple(gene_interactions['shape'])
)

index_choice = input("Enter index: ")
index_choice = int(index_choice)

start = adj_matrix.indptr[index_choice]
end = adj_matrix.indptr[index_choice + 1]

indices = adj_matrix.indices[start:end]
values = adj_matrix.data[start:end]

gene_names = bg_nonnull[bg_nonnull['matrix_index'].isin(indices)]
print(len(gene_names))
print(gene_names)
maxval = 0
maxindex = 0
for i, v in zip(indices, values):
    if v > maxval: maxval = v; maxindex = i
    #print(f"index: {i}, value: {v}, gene name: {gene_names[gene_names['matrix_index'] == i]['gene'].values[0]}")

print(f"Max value: {maxval}, at index: {maxindex}, gene name: {gene_names[gene_names['matrix_index'] == maxindex]['gene'].values[0]}")



22
          gene  matrix_index
1400     KIF14          1542
1950    COMMD1          2159
3352      MCM2          3655
4117   METTL14          4481
4746    KIF20A          5153
5472      CUL7          5945
6370     MEPCE          6916
8646      DKK3          9491
9417     HSPA8         10365
10263      CIT         11307
10974     DLST         12073
11428    KIF23         12569
11469      CSK         12614
11596     PRC1         12754
11841     GDE1         13025
11963    PHKG2         13156
12487    AURKB         13723
12804    KRT17         14072
13009   TRIM25         14285
14401  PLEKHA4         15798
15034   PARD6B         16509
16285   MAGEA3         17875
Max value: 1.0, at index: 1542, gene name: KIF14


In [ ]:
gene_variance = pd.read_csv('data/expression_variance.tsv', sep='\t', header=0)


gene_variance = gene_variance[gene_variance['sample'].isin(bg_nonnull['gene'])] # Filter by genes present in non-null csv


top_200_genes = gene_variance.nlargest(200, 'variance')['sample'].tolist()      # Get top 200 by variance


filtered_gene_expression = expression_data[top_200_genes + ['sample']]          # Filter expression data only to genes we want but keep sample id
filtered_gene_expression = filtered_gene_expression.set_index('sample')
print(filtered_gene_expression.shape)
print(filtered_gene_expression)
print(filtered_gene_expression['TCGA-3C-AAAU'])
filtered_index_df = pd.DataFrame({'gene':top_200_genes})

(981, 200)
Index(['KRT5', 'CEACAM5', 'KRT6A', 'XIST', 'CEACAM6', 'AGR2', 'KRT14', 'KRT6B', 'SLC34A2', 'KRT16',
       ...
       'SFTPA1', 'SPRR2D', 'ALB', 'MAGEA4', 'ST6GALNAC1', 'GGT6', 'SLC17A1', 'APOA2', 'EMX2', 'KLK2'],
      dtype='object', length=200)


KeyError: 'TCGA-3C-AAAU'

# Node Attention Layer

In [ ]:
class NodeAttentionLayer(nn.Module):
    def __init__(self, nodeDim, edgeDim, hidDim):
        super().__init__()

        self.node_linear = nn.Linear(nodeDim, hidDim)
        self.edge_linear = nn.Linear(edgeDim, hidDim)

        self.node_attention = nn.Linear(2*hidDim, 1)
        self.edge_attention = nn.Linear(2*hidDim, 1)

        self.leaky_relu = nn.LeakyReLU(0.05)

    def forward(self, x, edge_index, edge_attr):
        N = x.size(0)
        E = edge_attr.size(0)

        h = self.node_linear(x)             #[N, hidDim]
        g = self.edge_linear(edge_attr)     #[E, hidDim]

        # Gather structural neighborhood structural info
        i_idx, j_idx = edge_index
        h_i = h[i_idx]                      #[E, hidDim]
        h_j = h[j_idx]                      #[E, hidDim]
        g_k = g                             #[E, hidDim]

        # Node neighbor attention scores
        # e_ij = LeakyReLU( aᵀ · [h_i ∥ h_j] )
        e_ij = self.leaky_relu(self.node_attention(torch.cat([h_i, h_j], dim=1))).squeeze(-1)   # [E]

        # Normalize per-target-node i over its neighbors j ∈ N(i)
        a_n = softmax(e_ij, i_idx)      # [E]

        # Embedding aggregation
        x_nodes = scatter(a_n.unsqueeze(-1) * h_j, i_idx, dim=0, dim_size=N, reduce='mean')     # [N, hidDim]
        x_nodes = self.leaky_relu(x_nodes)

        # Edge neighbor attention scores
        # e_i⁽edge⁾ = LeakyReLU( bᵀ · [h_i ∥ g_k] )
        e_ie = self.leaky_relu(self.edge_attention(torch.cat([h_i, g_k], dim=1))).squeeze(-1)   # [E]

        # Normalize over edges k ∈ E(i)
        a_e = softmax(e_ie, i_idx)      # [E]

        # Embedding aggregation
        x_edges = scatter(a_e.unsqueeze(-1) * g_k, i_idx, dim=0, dim_size=N, reduce='mean')     # [N, hidDim]
        x_edges = self.leaky_relu(x_edges)

        # Concat and return
        return torch.cat([x_nodes, x_edges], dim=1)     # [N, 2*hidDim]

# Edge Attention Layer

In [ ]:
class EdgeAttentionLayer(nn.Module):
    def __init__(self, nodeDim, edgeDim, hidDim):
        super().__init__()

        self.node_linear = nn.Linear(nodeDim, hidDim)
        self.edge_linear = nn.Linear(edgeDim, hidDim)

        self.edge_attention = nn.Linear(2*hidDim, 1)
        self.node_attention = nn.Linear(2*hidDim, 1)

        self.leaky_relu = nn.LeakyReLU(0.05)

    def forward(self, x, edge_index, edge_attr):
        N = x.size(0)
        E = edge_attr.size(0)

        h = self.node_linear(x)             #[N, hidDim]
        g = self.edge_linear(edge_attr)     #[E, hidDim]

        # Gather edge endpoints
        u_idx, v_idx = edge_index           # Both [E]
        edge_idx = torch.arange(E, device=g.device)

        # Replicate for 2 endpoints
        edge_idx_rep = edge_idx.repeat(2)
        node_feats = torch.cat([h[u_idx], h[v_idx]], dim=0)
        edge_feats_rep = g[edge_idx_rep]

        # Node attention (need better explanation of edge prep)
        e_en = self.leaky_relu(self.node_attention(torch.cat([edge_feats_rep, node_feats], dim=1))).squeeze(-1)
        B_n = softmax(e_en, edge_idx_rep)
        # Embedding aggregation
        e_from_nodes = scatter(B_n.unsqueeze(-1) * node_feats, edge_idx_rep, dim=0, dim_size=E, reduce='mean')
        e_from_nodes = self.leaky_relu(e_from_nodes)

        # Build edge-edge adjacency with line_graph
        edge_edge_index, _ = line_graph(edge_index, force_undirected=False)     # [2, #pairs]

        row_e, col_e = edge_edge_index

        # Edge attention (need better explanation of edge setup w/ line_graph)
        e_ee = self.leaky_relu(self.edge_attention(torch.cat([g[row_e], g[col_e]], dim=1))).squeeze(-1)
        B_e = softmax(e_ee, row_e)
        # Embedding aggregation
        e_from_edges = scatter(B_e.unsqueeze(-1) * g[col_e], row_e, dim=0, dim_size=E, reduce='mean')
        e_from_edges = self.leaky_relu(e_from_edges)

        # Concat and return
        return torch.cat([e_from_nodes, e_from_edges], dim=1)

# Full Network

In [ ]:
class HierarchicalDualAttentionModel(nn.Module):
    

SyntaxError: unexpected EOF while parsing (3039522207.py, line 2)